In [1]:
import pandas as pd
import numpy as np

In [13]:
# First, let's add the candidate data into congresspeople

# Read in the data
candidates = pd.read_csv('data/candidates/candidates.csv', encoding='utf-8')
congresspeople = pd.read_csv('data/congresspeople/congresspeople.csv', encoding='utf-8')

In [14]:
# Filter out the candidates that are not congresspeople
candidates = candidates[candidates['DS_CARGO'] == 'DEPUTADO FEDERAL']

# Replace #NE with np.nan
candidates['DS_COR_RACA'] = candidates['DS_COR_RACA'].replace('#NE', np.nan)
candidates['DS_COR_RACA'] = candidates['DS_COR_RACA'].replace('NA', np.nan)
candidates['DS_COR_RACA'] = candidates['DS_COR_RACA'].replace('#NE#', np.nan)

# For the candidates with np.nan, look for the next ANO_ELEICAO and use that
candidates['DS_COR_RACA'] = candidates.groupby('NM_CANDIDATO')['DS_COR_RACA'].fillna(method='bfill')
candidates['DS_COR_RACA']

# Filtering important columns
candidates = candidates[['ANO_ELEICAO', 'DS_OCUPACAO', 'NR_CPF_CANDIDATO', 
                         'DS_GENERO', 'DS_GRAU_INSTRUCAO', 'DS_ESTADO_CIVIL', 'DS_COR_RACA',]]

# Rename columns
candidates.rename({'NR_CPF_CANDIDATO': "cpf", 'DS_OCUPACAO': 'occupation',
                   'DS_GENERO': 'gender', 'DS_GRAU_INSTRUCAO': 'education',
                   'DS_ESTADO_CIVIL': 'marital_status', 'DS_COR_RACA': 'ethnicity',
                   'ANO_ELEICAO': 'election_year'
                   }, axis=1, inplace=True)

candidates['election_year'] = candidates['election_year'].astype('int64')


# Adding election year to congresspeople and converting cpf to string
election_year = {57: 2022, 56: 2018, 55: 2014,
                 54: 2010, 53: 2006, 52: 2002,
                 51: 1998}
congresspeople['election_year'] = congresspeople['idLegislatura'].apply(lambda x: election_year[x])
congresspeople['cpf'] = congresspeople['cpf'].astype(str)

# Drop duplicated congresspeople, keeping the first one. Each congressperson will have only one candidate per election year
congresspeople = congresspeople.drop_duplicates(subset=['cpf', 'election_year'], keep='first')

In [15]:
# Merge the data, by year and cpf
congresspeople = congresspeople.merge(candidates, on=['cpf', 'election_year'], how='left')
congresspeople

,id,nomeCivil,nomeEleitoral,ufNascimento,escolaridade,sexo,cpf,redeSocial,siglaUf,siglaPartido,idLegislatura,election_year,occupation,gender,education,marital_status,ethnicity
0,65551,RICARDO WAGNER DE CARVALHO LAGO,WAGNER LAGO,MA,Superior,M,1706047304,[],MA,PDT,52,2002,ADVOGADO,MASCULINO,SUPERIOR COMPLETO,CASADO(A),NaN
1,139285,LÍDICE DA MATA E SOUZA,Lídice da Mata,BA,Superior,F,14672049515,"['https://twitter.com/lidicedamata', 'https://...",BA,PSB,57,2022,DEPUTADO,FEMININO,SUPERIOR COMPLETO,DIVORCIADO(A),PARDA
2,139285,LÍDICE DA MATA E SOUZA,Lídice da Mata,BA,Superior,F,14672049515,"['https://twitter.com/lidicedamata', 'https://...",BA,PSB,53,2006,DEPUTADO,FEMININO,SUPERIOR COMPLETO,DIVORCIADO(A),PARDA
3,139285,LÍDICE DA MATA E SOUZA,Lídice da Mata,BA,Superior,F,14672049515,"['https://twitter.com/lidicedamata', 'https://...",BA,PSB,56,2018,SENADOR,FEMININO,SUPERIOR COMPLETO,DIVORCIADO(A),PARDA
4,196357,DEJORGE PATRICIO DA SILVA,DEJORGE PATRÍCIO,RJ,Ensino Médio,M,3726063773,[],RJ,PRB,55,2014,VEREADOR,MASCULINO,ENSINO MÉDIO COMPLETO,SOLTEIRO(A),PARDA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,163831,FERNANDO ANTÔNIO CECILIANO JORDÃO,FERNANDO JORDÃO,RJ,Superior,M,49752839720,[],RJ,PMDB,54,2010,EMPRESÁRIO,MASCULINO,SUPERIOR COMPLETO,CASADO(A),NaN
4406,73720,César Antonio de Souza,CESAR SOUZA,SC,Secundário,M,28951107949,[],SC,PSD,55,2014,EMPRESÁRIO,MASCULINO,ENSINO MÉDIO INCOMPLETO,DIVORCIADO(A),BRANCA
4407,180214,RAIMUNDO DINIZ ARAÚJO,Pastor Diniz,RR,Superior,M,65822960278,[],RR,UNIÃO,57,2022,PSICÓLOGO,MASCULINO,SUPERIOR COMPLETO,CASADO(A),PARDA
4408,172029,FRANCISCO DAS CHAGAS FRANCILINO,FRANCISCO CHAGAS,SE,Superior,M,76555283815,[],SP,PT,54,2010,VEREADOR,MASCULINO,SUPERIOR COMPLETO,DIVORCIADO(A),BRANCA


,id,nomeCivil,nomeEleitoral,ufNascimento,escolaridade,sexo,cpf,redeSocial,siglaUf,siglaPartido,idLegislatura,election_year,occupation,gender,education,marital_status,ethnicity
6,196357,DEJORGE PATRICIO DA SILVA,DEJORGE PATRÍCIO,RJ,Ensino Médio,M,3726063773,[],RJ,PRB,55,2014,VEREADOR,MASCULINO,ENSINO MÉDIO COMPLETO,SOLTEIRO(A),PARDA
7,196357,DEJORGE PATRICIO DA SILVA,DEJORGE PATRÍCIO,RJ,Ensino Médio,M,3726063773,[],RJ,PR,55,2014,VEREADOR,MASCULINO,ENSINO MÉDIO COMPLETO,SOLTEIRO(A),PARDA
9,204498,OLIVAL HENRIQUE MARQUES DE SOUZA,Olival Marques,PA,Superior,M,66514010253,"['https://twitter.com/OlivalMarquesOf', 'https...",PA,DEM,56,2018,OUTROS,MASCULINO,SUPERIOR COMPLETO,CASADO(A),PARDA
10,204498,OLIVAL HENRIQUE MARQUES DE SOUZA,Olival Marques,PA,Superior,M,66514010253,"['https://twitter.com/OlivalMarquesOf', 'https...",PA,UNIÃO,56,2018,OUTROS,MASCULINO,SUPERIOR COMPLETO,CASADO(A),PARDA
12,73758,FLÁVIO AUGUSTO COELHO DERZI,FLÁVIO DERZI,MS,Superior,M,29651808772,[],MS,PMDB,51,1998,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7081,221157,RUI GODINHO DA MOTA,Rui Godinho,RJ,Superior,M,8195182712,[],SC,UNIÃO,56,2018,NaN,NaN,NaN,NaN,NaN
7082,221157,RUI GODINHO DA MOTA,Rui Godinho,RJ,Superior,M,8195182712,[],SC,PSL,56,2018,NaN,NaN,NaN,NaN,NaN
7087,73714,José Reinaldo Carneiro Tavares,JOSÉ REINALDO,MA,Superior,M,138754349,[],MA,PSDB,55,2014,ENGENHEIRO,MASCULINO,SUPERIOR COMPLETO,SEPARADO(A) JUDICIALMENTE,BRANCA
7088,73714,José Reinaldo Carneiro Tavares,JOSÉ REINALDO,MA,Superior,M,138754349,[],MA,PSB,55,2014,ENGENHEIRO,MASCULINO,SUPERIOR COMPLETO,SEPARADO(A) JUDICIALMENTE,BRANCA


In [9]:
duplicated[duplicated['idLegislatura'] == 57]

,id,nomeCivil,nomeEleitoral,ufNascimento,escolaridade,sexo,cpf,redeSocial,siglaUf,siglaPartido,idLegislatura,election_year,occupation,gender,education,marital_status,ethnicity
1177,74270,GILBERTO NASCIMENTO SILVA,Gilberto Nascimento,SP,Superior,M,64817407891,[],SP,PSD,57,2022,DEPUTADO,MASCULINO,SUPERIOR COMPLETO,CASADO(A),BRANCA
1181,74270,GILBERTO NASCIMENTO SILVA,Gilberto Nascimento,SP,Superior,M,64817407891,[],SP,PSC,57,2022,DEPUTADO,MASCULINO,SUPERIOR COMPLETO,CASADO(A),BRANCA
3181,141459,JOÃO DA SILVA MAIA,João Maia,PB,Superior,M,6662978191,[],RN,PP,57,2022,DEPUTADO,MASCULINO,SUPERIOR COMPLETO,CASADO(A),BRANCA
3185,141459,JOÃO DA SILVA MAIA,João Maia,PB,Superior,M,6662978191,[],RN,PL,57,2022,DEPUTADO,MASCULINO,SUPERIOR COMPLETO,CASADO(A),BRANCA
3992,160518,WELITON FERNANDES PRADO,Weliton Prado,MG,Superior,M,84719850634,['https://www.instagram.com/weliton.prado'],MG,SOLIDARIEDADE,57,2022,FOTÓGRAFO E ASSEMELHADOS,MASCULINO,SUPERIOR COMPLETO,SOLTEIRO(A),PARDA
3994,160518,WELITON FERNANDES PRADO,Weliton Prado,MG,Superior,M,84719850634,['https://www.instagram.com/weliton.prado'],MG,PROS,57,2022,FOTÓGRAFO E ASSEMELHADOS,MASCULINO,SUPERIOR COMPLETO,SOLTEIRO(A),PARDA
4793,227275,TACILLA MARIANA CARVALHO SILVA,Mariana Carvalho,MA,NaN,F,64086305372,[],MA,PODE,57,2022,ADVOGADO,FEMININO,SUPERIOR COMPLETO,CASADO(A),PARDA
5087,178881,ALUISIO GUIMARAES MENDES FILHO,Aluisio Mendes,MG,Superior,M,66746485749,"['https://twitter.com/aluisiomendesma', 'https...",MA,REPUBLICANOS,57,2022,DEPUTADO,MASCULINO,SUPERIOR COMPLETO,CASADO(A),BRANCA
5092,178881,ALUISIO GUIMARAES MENDES FILHO,Aluisio Mendes,MG,Superior,M,66746485749,"['https://twitter.com/aluisiomendesma', 'https...",MA,PSC,57,2022,DEPUTADO,MASCULINO,SUPERIOR COMPLETO,CASADO(A),BRANCA
5785,220607,MAX RODRIGUES LEMOS,Max Lemos,RJ,Superior,M,75061600720,"['https://twitter.com/maxlemosoficial', 'https...",RJ,SOLIDARIEDADE,57,2022,DEPUTADO,MASCULINO,SUPERIOR COMPLETO,CASADO(A),PARDA
